# Data loading 
Next cells are used to load the data we need.

In [1]:
import os

os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1

import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
sys.path.append('../../')


In [2]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('../data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [3]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [4]:
ICM_df = pd.read_csv('../data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [5]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [6]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [7]:
from MatrixFactorization.IALSRecommender_implicit import IALSRecommender_implicit

URM_train_ICM_all_IALS = sps.vstack([URM_all*(1-0.51668), ICM_all.T*0.51668])
recommender_IALS = IALSRecommender_implicit(URM_train_ICM_all_IALS)
recommender_IALS.fit(n_factors = 601, regularization = 0.8715, iterations=83, num_threads=4)

Recommender_Base_Class: URM Detected 2 (0.01 %) cold users.


In [8]:
from GraphBased.RP3betaRecommender import RP3betaRecommender

URM_train_ICM_all_RP3 = sps.vstack([URM_all, ICM_all.T])
recommender_rp3Beta = RP3betaRecommender(URM_train_ICM_all_RP3)
recommender_rp3Beta.fit(topK=350, alpha = 0.3120, beta=0.1979, implicit = False) #0.19798705532101593
recommender_rp3Beta.URM_train = URM_train

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.


In [9]:
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

URM_train_ICM_all_SLIM = sps.vstack([URM_all*(1-0.421), ICM_all.T*0.421])
recommender_SLIM = SLIMElasticNetRecommender(URM_train_ICM_all_SLIM)
recommender_SLIM.fit(topK=996, alpha = 0.000182, l1_ratio = 4.459e-06)
recommender_rp3Beta.W_sparse.data = np.power(recommender_rp3Beta.W_sparse.data, 0.8546)

SLIMElasticNetRecommender: URM Detected 2 (0.01 %) cold users.


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0017660893499851227, tolerance: 0.0008219736628234386
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0016531796427443624, tolerance: 0.0005707714008167386
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.004204806871712208, tolerance: 0.0010425166692584753
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs

SLIMElasticNetRecommender: Processed 3023 ( 11.64% ) in 5.00 min. Items per second: 10.07


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.007234418764710426, tolerance: 0.002652232302352786
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0003839170385617763, tolerance: 0.00015649839770048857
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0019544092938303947, tolerance: 0.0005939972470514476
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs

SLIMElasticNetRecommender: Processed 6087 ( 23.43% ) in 10.00 min. Items per second: 10.14


...

SLIMElasticNetRecommender: Processed 22164 ( 85.33% ) in 35.01 min. Items per second: 10.55


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0004205565492156893, tolerance: 0.0002552639052737504
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00033482498838566244, tolerance: 0.00023872620658949018
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00042294865124858916, tolerance: 0.0002792770101223141
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\

SLIMElasticNetRecommender: Processed 25653 ( 98.76% ) in 40.01 min. Items per second: 10.69


C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.464852809906006, tolerance: 0.007426317315548658
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0031207562424242496, tolerance: 0.0007116304477676749
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my_env\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:512: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0007056495524011552, tolerance: 0.00022016357979737222
  model = cd_fast.sparse_enet_coordinate_descent(
C:\Users\matti\anaconda3\envs\my

SLIMElasticNetRecommender: Processed 25975 ( 100.00% ) in 40.50 min. Items per second: 10.69


In [10]:
from KNN.ItemKNNScoresHybridNRecommender import ItemKNNScoresHybridNRecommender
URM_train_ICM_all = sps.vstack([URM_all, ICM_all.T])

hybrid = ItemKNNScoresHybridNRecommender(URM_train_ICM_all, [recommender_IALS,recommender_rp3Beta,recommender_SLIM], 3)
hybrid.fit([0.15447,0.6287, 0.2854])
#evaluator_validation.evaluateRecommender(hybrid)

ItemKNNScoresHybridNRecommender: URM Detected 2 (0.01 %) cold users.
number of recommenders: 3


In [11]:
test_users = pd.read_csv('../data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [12]:
user_id = test_users['user_id']
recommendations = hybrid.recommend(user_id,cutoff = 10)


In [13]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
print(len(recommendations))

test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

test_users
test_users.to_csv('submission.csv', index=False)


7944
